## Fetch Stations form SBB API

In [ ]:
from IPython.display import clear_output
import os
import sqlite3
import requests
import pandas as pd
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Database file
DATABASE_FILE = "locations.db"

# Function to initialize the database
def initialize_database():
    conn = sqlite3.connect(DATABASE_FILE)
    cursor = conn.cursor()

    # Create table for locations if it doesn't exist
    cursor.execute(
        """
        CREATE TABLE IF NOT EXISTS locations (
            id TEXT PRIMARY KEY,
            name TEXT,
            coordinate_x REAL,
            coordinate_y REAL,
            distance REAL
        )
        """
    )
    conn.commit()
    conn.close()

# Function to check if a city already has data in the database
def city_exists_in_db(city):
    conn = sqlite3.connect(DATABASE_FILE)
    cursor = conn.cursor()

    cursor.execute(
        """
        SELECT 1 FROM locations WHERE name = ? LIMIT 1
        """,
        (city,)
    )
    exists = cursor.fetchone() is not None
    conn.close()
    return exists

# Function to insert data into the database
def insert_into_database(data):
    conn = sqlite3.connect(DATABASE_FILE)
    cursor = conn.cursor()

    cursor.executemany(
        """
        INSERT OR IGNORE INTO locations (id, name, coordinate_x, coordinate_y, distance)
        VALUES (?, ?, ?, ?, ?)
        """,
        data,
    )
    conn.commit()
    conn.close()

# Function to fetch location data for multiple cities
def fetch_locations(cities):
    """Fetches location data for a list of cities and saves them to the database."""
    for idx, city in enumerate(cities, 1):
        clear_output(wait=True)  # Clear the output
        print(f"Processing city {idx}/{len(cities)}: {city}")

        if city_exists_in_db(city):
            print(f"Data for {city} already exists in the database. Skipping.")
            continue

        url = f"http://transport.opendata.ch/v1/locations?query={city}"
        try:
            response = requests.get(url)

            if response.status_code == 200:
                data = response.json()
                stations = data.get("stations", [])

                # Prepare data for database insertion
                db_data = [
                    (
                        station.get("id", ""),
                        station.get("name", ""),
                        station.get("coordinate", {}).get("x", None),
                        station.get("coordinate", {}).get("y", None),
                        station.get("distance", None),
                    )
                    for station in stations
                ]

                insert_into_database(db_data)
            else:
                print(f"Error fetching data for {city}: {response.status_code}")
        except Exception as e:
            print(f"An error occurred for {city}: {e}")

# Function to read data from the database for analysis
def fetch_data_from_db():
    conn = sqlite3.connect(DATABASE_FILE)
    query = "SELECT * FROM locations"
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

if __name__ == "__main__":
    # Initialize database
    initialize_database()

    # List of Swiss cities near Zurich
     
    cities_near_zurich = [
    # Zurich neighborhoods
    "Zurich", "Altstetten", "Albisrieden", "Oberstrass", "Unterstrass", "Seebach", "Affoltern", "Höngg",
    "Wipkingen", "Wiedikon", "Enge", "Wollishofen", "Leimbach", "Fluntern", "Hottingen", "Hirslanden", "Mühlebach",

    # Municipalities in Zurich Canton
    "Adlikon", "Adliswil", "Aesch", "Aeugst am Albis", "Affoltern am Albis", "Altikon", "Andelfingen",
    "Bachenbülach", "Bachs", "Bäretswil", "Bassersdorf", "Bauma", "Benken", "Berg am Irchel", "Birmensdorf",
    "Bonstetten", "Boppelsen", "Brütten", "Bubikon", "Buch am Irchel", "Buchs", "Bülach", "Dachsen", "Dägerlen",
    "Dällikon", "Dänikon", "Dättlikon", "Dielsdorf", "Dietikon", "Dietlikon", "Dinhard", "Dorf", "Dübendorf",
    "Dürnten", "Egg", "Eglisau", "Elgg", "Ellikon an der Thur", "Elsau", "Embrach", "Erlenbach", "Fällanden",
    "Fehraltorf", "Feuerthalen", "Fischenthal", "Flaach", "Flurlingen", "Freienstein-Teufen", "Geroldswil",
    "Glattfelden", "Gossau", "Greifensee", "Grüningen", "Hagenbuch", "Hausen am Albis", "Hedingen", "Henggart",
    "Herrliberg", "Hettlingen", "Hinwil", "Hittnau", "Hochfelden", "Hombrechtikon", "Horgen", "Höri", "Hüntwangen",
    "Hüttikon", "Illnau-Effretikon", "Kappel am Albis", "Kilchberg", "Kleinandelfingen", "Kloten", "Knonau",
    "Küsnacht", "Langnau am Albis", "Laufen-Uhwiesen", "Lindau", "Lufingen", "Männedorf", "Marthalen", "Maschwanden",
    "Maur", "Meilen", "Mettmenstetten", "Mönchaltorf", "Neerach", "Neftenbach", "Niederglatt", "Niederhasli",
    "Niederweningen", "Nürensdorf", "Oberembrach", "Oberengstringen", "Oberglatt", "Oberrieden", "Oberweningen",
    "Obfelden", "Oetwil am See", "Oetwil an der Limmat", "Opfikon", "Ossingen", "Otelfingen", "Ottenbach", "Pfäffikon",
    "Pfungen", "Rafz", "Regensberg", "Regensdorf", "Rheinau", "Richterswil", "Rickenbach", "Rifferswil", "Rikon im Tösstal",
    "Rorbas", "Rümlang", "Rüschlikon", "Russikon", "Rüti", "Schlatt", "Schleinikon", "Schlieren", "Schöfflisdorf",
    "Schwerzenbach", "Seegräben", "Seuzach", "Stadel", "Stäfa", "Stallikon", "Stammheim", "Steinmaur", "Thalheim an der Thur",
    "Thalwil", "Trüllikon", "Truttikon", "Turbenthal", "Uetikon am See", "Uitikon", "Unterengstringen", "Urdorf", "Uster",
    "Volken", "Volketswil", "Wädenswil", "Wald", "Wallisellen", "Wangen-Brüttisellen", "Wasterkingen", "Weiach",
    "Weiningen", "Weisslingen", "Wettswil am Albis", "Wetzikon", "Wiesendangen", "Wil", "Wila", "Wildberg", "Winkel",
    "Winterthur", "Zell", "Zollikon", "Zumikon", 

    # Surrounding regions (Aargau, Zug, Schwyz, etc.)
    "Bremgarten", "Spreitenbach", "Wohlen", "Wettingen", "Baar", "Cham", "Jona", "Illnau", "Schindellegi",
    "Oberägeri", "Unterägeri", "Freienbach", "Pfäffikon SZ", "Rapperswil-Jona", "Tuggen", "Wollerau", "Zug",
    "Horgenberg", "Schwyz", "Richterswil", "Sattel", "Arth", "Goldau", "Steinhausen", "Cham", "Rotkreuz",
    
    # Other villages and named areas
    "Adlisberg", "Gockhausen", "Stettbach", "Itschnach", "Hinteregg", "Forch", "Schönenberg", "Hütten", "Küsnachterberg",
    "Wängi", "Berikon", "Dietwil", "Einsiedeln", "Biberbrugg", "Wädenswilberg", "Thalwilberg", "Frauenfeld"
]




    # Fetch and save location data
    fetch_locations(cities_near_zurich)

    # Fetch data for analysis
    print("Fetching data for analysis...")
    df = fetch_data_from_db()
    print(df.head())  # Display the first few rows of the data

    print("Data fetching and saving completed.")


## Clean Results

In [ ]:
import os
import sqlite3
import pandas as pd

# Database file
DATABASE_FILE = "locations.db"

# Function to clean and transform data in the database
def clean_and_transform_data():
    conn = sqlite3.connect(DATABASE_FILE)
    
    # Load the data into a pandas DataFrame
    query = "SELECT * FROM locations"
    df = pd.read_sql_query(query, conn)

    # Remove rows where the 'id' column is empty
    df = df.dropna(subset=['id'])

    # Split the 'name' column into two parts at the comma
    df[['name_part_1', 'name_part_2']] = df['name'].str.split(',', n=1, expand=True)

    # Replace NaN in 'name_part_2' with an empty string
    df['name_part_2'] = df['name_part_2'].fillna('')

    # Drop the original 'name' column if not needed
    df = df.drop(columns=['name'])

    # Save the cleaned and transformed data back into the database (optional table)
    cursor = conn.cursor()
    cursor.execute("DROP TABLE IF EXISTS cleaned_locations")
    cursor.execute(
        """
        CREATE TABLE cleaned_locations (
            id TEXT PRIMARY KEY,
            coordinate_x REAL,
            coordinate_y REAL,
            distance REAL,
            name_part_1 TEXT,
            name_part_2 TEXT
        )
        """
    )

    # Insert cleaned data back into the database
    df.to_sql('cleaned_locations', conn, if_exists='replace', index=False)

    conn.commit()
    conn.close()
    print("Cleaned data has been saved to the 'cleaned_locations' table in the database.")

if __name__ == "__main__":
    clean_and_transform_data()


## Combine Stations

In [ ]:
import os
import sqlite3
import pandas as pd

# Database file
DATABASE_FILE = "locations.db"

def combine_data():
    try:
        conn = sqlite3.connect(DATABASE_FILE)

        # Load the cleaned data into a pandas DataFrame
        query = "SELECT * FROM cleaned_locations"
        df = pd.read_sql_query(query, conn)

        if df.empty:
            print("No data found in 'cleaned_locations' table.")
            return

        # Define a function to select the row with the shortest name_part_2
        def select_shortest_name_part_2(group):
            return group.loc[group['name_part_2'].str.len().idxmin()]

        # Group by 'name_part_1' and apply the function
        combined_df = (
            df.groupby('name_part_1', group_keys=False)
            .apply(select_shortest_name_part_2)
            .reset_index(drop=True)
        )

        # Create a new table for combined data
        cursor = conn.cursor()
        cursor.execute("DROP TABLE IF EXISTS combined_locations")
        cursor.execute(
            """
            CREATE TABLE combined_locations (
                id TEXT PRIMARY KEY,
                name_part_1 TEXT,
                name_part_2 TEXT,
                coordinate_x REAL,
                coordinate_y REAL,
                distance REAL
            )
            """
        )

        # Insert combined data back into the database
        combined_df.to_sql('combined_locations', conn, if_exists='replace', index=False)

        conn.commit()
        conn.close()
        print("Combined data has been saved to the 'combined_locations' table in the database.")

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    combine_data()


In [ ]:
import os
import sqlite3
import time
import random
import requests
import pandas as pd
from IPython.display import clear_output
import json

# Database file
DATABASE_FILE = "locations.db"

# Initialize the transport results table
def initialize_transport_table():
    conn = sqlite3.connect(DATABASE_FILE)
    cursor = conn.cursor()
    cursor.execute(
        """
        CREATE TABLE IF NOT EXISTS transport_results_2 (
            from_station TEXT,
            to_station TEXT,
            min_duration INTEGER,
            max_duration INTEGER,
            transfers INTEGER,
            connections INTEGER,
            legs TEXT,
            PRIMARY KEY (from_station, to_station)
        )
        """
    )
    conn.commit()
    conn.close()

# Check if transport data exists in the database
def transport_data_exists(from_station, to_station):
    conn = sqlite3.connect(DATABASE_FILE)
    cursor = conn.cursor()
    cursor.execute(
        """
        SELECT 1 FROM transport_results_2 WHERE from_station = ? AND to_station = ? LIMIT 1
        """,
        (from_station, to_station),
    )
    exists = cursor.fetchone() is not None
    conn.close()
    return exists

# Save a single transport result to the database
def save_single_transport_data(from_station, to_station, min_duration, max_duration, transfers, connections, legs):
    # Convert duration from seconds to minutes
    min_duration_in_minutes = min_duration // 60  # Integer division to avoid fractions
    max_duration_in_minutes = max_duration // 60
    conn = sqlite3.connect(DATABASE_FILE)
    cursor = conn.cursor()
    cursor.execute(
        """
        INSERT OR REPLACE INTO transport_results_2 
        (from_station, to_station, min_duration, max_duration, transfers, connections, legs)
        VALUES (?, ?, ?, ?, ?, ?, ?)
        """,
        (from_station, to_station, min_duration_in_minutes, max_duration_in_minutes, transfers, connections, json.dumps(legs)),  # Added legs
    )
    conn.commit()
    conn.close()


# Process transport data with API integration
def process_transport_data(to_station, delay=1.5, travel_date="12/17/2024", travel_time="08:00", num=4):
    try:
        # Fetch starting stations
        conn = sqlite3.connect(DATABASE_FILE)
        query = "SELECT DISTINCT name_part_1 FROM combined_locations"
        from_stations = pd.read_sql_query(query, conn)['name_part_1'].dropna().tolist()
        conn.close()

        if not from_stations:
            print("No starting stations found in the database.")
            return

        session = requests.Session()

        for index, from_station in enumerate(from_stations, start=1):
            
            print(f"Processing {index}/{len(from_stations)}: From '{from_station}' to '{to_station}'...")

            # Skip if the data already exists
            if transport_data_exists(from_station, to_station):
                print(f"Data already exists for '{from_station}' to '{to_station}'. Skipping.")
                continue

            # Simulate delay to respect API rate limits
            #time.sleep(random.uniform(delay, delay + 1))

            # API request
            params = {
                "from": from_station,
                "to": to_station,
                "date": travel_date,
                "time": travel_time,
                "num": num,
            }
            url = "https://search.ch/timetable/api/route.json"
            response = session.get(url, params=params)

            if response.status_code == 200:
                data = response.json()
                if data.get('connections'):
                    # Extract details from the response
                    min_duration = data.get("min_duration", 0)
                    max_duration = data.get("max_duration", 0)
                    connections = len(data.get("connections", []))
                    transfers = len(data["connections"][0].get("legs", [])) - 1

                    # Extract detailed legs information
                    legs = [
                        {
                            "type": leg.get("type"),
                            "line": leg.get("line"),
                            "departure": leg.get("departure"),
                            "arrival": leg.get("arrival"),
                            "stops": [
                                stop.get("name") for stop in leg.get("stops", [])
                            ] if leg.get("stops") else []
                        }
                        for leg in data["connections"][0].get("legs", [])
                    ]

                    # Save the data immediately to the database
                    save_single_transport_data(from_station, to_station, min_duration, max_duration, transfers, connections, legs)
                    print(f"Saved data for '{from_station}' to '{to_station}'.")

                else:
                    print(f"No connections found for '{from_station}' to '{to_station}'.")
            else:
                print(f"Error fetching data for '{from_station}' to '{to_station}': {response.status_code}")

        print("\nProcessing complete. All data has been saved.")

    except Exception as e:
        print(f"An unexpected error occurred: {e}")

if __name__ == "__main__":
    # Initialize transport table
    initialize_transport_table()

    # Destination station
    to_station = "Winterthur, Hauptbahnhof"

    # Debugging: Check combined_locations table content
    print("Checking combined_locations table...")
    conn = sqlite3.connect(DATABASE_FILE)
    query = "SELECT DISTINCT name_part_1 FROM combined_locations"
    from_stations = pd.read_sql_query(query, conn)['name_part_1'].dropna().tolist()
    conn.close()

    if not from_stations:
        print("No starting stations found in the combined_locations table.")
    else:
        print(f"Loaded {len(from_stations)} starting stations:")
        print(from_stations[:10])  # Print first 10 stations for verification

    process_transport_data(to_station)


## GET PLZ for all Stations

In [ ]:
import sqlite3
import requests
import pandas as pd
import time
from dotenv import load_dotenv
import os
from IPython.display import clear_output

# Load environment variables
load_dotenv()
API_KEY = os.getenv("API_KEY")

if not API_KEY:
    raise ValueError("API key not found. Please check your .env file and environment setup.")

# Define the LocationIQ Reverse Geocoding API
API_URL = "https://us1.locationiq.com/v1/reverse"

# Database file
DATABASE_FILE = "locations.db"

# Function to initialize the PLZ table
def initialize_plz_table():
    conn = sqlite3.connect(DATABASE_FILE)
    cursor = conn.cursor()
    cursor.execute(
        """
        CREATE TABLE IF NOT EXISTS plz_data (
            coordinate_x REAL,
            coordinate_y REAL,
            plz TEXT,
            PRIMARY KEY (coordinate_x, coordinate_y)
        )
        """
    )
    conn.commit()
    conn.close()

# Function to check if PLZ exists in the database
def plz_exists(lat, lon):
    conn = sqlite3.connect(DATABASE_FILE)
    cursor = conn.cursor()
    cursor.execute(
        """
        SELECT plz FROM plz_data WHERE coordinate_x = ? AND coordinate_y = ? LIMIT 1
        """,
        (lat, lon),
    )
    result = cursor.fetchone()
    conn.close()
    return result[0] if result else None

# Function to save PLZ data to the database
def save_plz_data(lat, lon, plz):
    conn = sqlite3.connect(DATABASE_FILE)
    cursor = conn.cursor()
    cursor.execute(
        """
        INSERT OR REPLACE INTO plz_data (coordinate_x, coordinate_y, plz)
        VALUES (?, ?, ?)
        """,
        (lat, lon, plz),
    )
    conn.commit()
    conn.close()

# Function to query the reverse geocoding API
def query_reverse_geocoding(lat, lon):
    params = {
        "key": API_KEY,
        "lat": lat,
        "lon": lon,
        "format": "json",
    }
    response = requests.get(API_URL, params=params)

    if response.status_code == 200:
        result = response.json()
        address = result.get("address", {})
        return address.get("postcode", "N/A")
    else:
        print(f"Error for coordinates ({lat}, {lon}): {response.status_code}")
    return "N/A"

# Function to process the data with progress updates
def process_and_query_plz():
    conn = sqlite3.connect(DATABASE_FILE)
    query = "SELECT * FROM combined_locations"
    df = pd.read_sql_query(query, conn)
    conn.close()

    # Check if required columns exist
    if not {'coordinate_x', 'coordinate_y'}.issubset(df.columns):
        print("Error: The database does not contain 'coordinate_x' or 'coordinate_y' columns.")
        return

    # Prepare a list to store PLZ values
    plz_list = []

    for index, row in df.iterrows():
        clear_output(wait=True)  # Clear output to reduce clutter
        lat, lon = row["coordinate_x"], row["coordinate_y"]
        print(f"Processing {index + 1}/{len(df)}: Coordinates ({lat}, {lon})")

        # Check if PLZ already exists in the database
        existing_plz = plz_exists(lat, lon)
        if existing_plz:
            print(f"PLZ for ({lat}, {lon}) found in database: {existing_plz}")
            plz_list.append(existing_plz)
        else:
            # Query the reverse geocoding API
            plz = query_reverse_geocoding(lat, lon)
            save_plz_data(lat, lon, plz)
            plz_list.append(plz)

            # Wait 1 second between requests to respect rate limits
            time.sleep(1)

    # Append the PLZ column to the DataFrame
    df["plz"] = plz_list

    # Save the updated DataFrame back to the database
    conn = sqlite3.connect(DATABASE_FILE)
    df.to_sql("combined_locations_with_plz", conn, if_exists="replace", index=False)
    conn.close()

    print("\nPLZ data has been updated and saved to the database.")

if __name__ == "__main__":
    # Initialize the PLZ table
    initialize_plz_table()

    # Process data and query PLZ
    process_and_query_plz()


In [ ]:
import os
import sqlite3
import time
import random
import requests
import re
import urllib.parse
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from IPython.display import clear_output

# Load environment variables
load_dotenv()

# Database file
DATABASE_FILE = "locations.db"

# Initialize the listings table
def initialize_listings_table():
    conn = sqlite3.connect(DATABASE_FILE)
    cursor = conn.cursor()
    cursor.execute(
        """
        CREATE TABLE IF NOT EXISTS listings (
            plz TEXT,
            maxprice INTEGER,
            RootPropertyType INTEGER,
            listing_count INTEGER,
            PRIMARY KEY (plz, maxprice, RootPropertyType)
        )
        """
    )
    conn.commit()
    conn.close()

# Initialize the error logs table
def initialize_error_logs_table():
    conn = sqlite3.connect(DATABASE_FILE)
    cursor = conn.cursor()
    cursor.execute(
        """
        CREATE TABLE IF NOT EXISTS error_logs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            plz TEXT,
            maxprice INTEGER,
            RootPropertyType INTEGER,
            http_code INTEGER,
            url TEXT,
            timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
        )
        """
    )
    conn.commit()
    conn.close()

# Save error logs to the database
def save_error_log(plz, maxprice, RootPropertyType, http_code, url):
    conn = sqlite3.connect(DATABASE_FILE)
    cursor = conn.cursor()
    cursor.execute(
        """
        INSERT INTO error_logs (plz, maxprice, RootPropertyType, http_code, url)
        VALUES (?, ?, ?, ?, ?)
        """,
        (plz, maxprice, RootPropertyType, http_code, url),
    )
    conn.commit()
    conn.close()

# Fetch PLZs from the database
def fetch_plz_from_db():
    conn = sqlite3.connect(DATABASE_FILE)
    query = "SELECT DISTINCT plz FROM plz_data"
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df['plz'].tolist()

# Check if listing data exists in the database
def listing_exists(plz, maxprice, RootPropertyType):
    conn = sqlite3.connect(DATABASE_FILE)
    cursor = conn.cursor()
    cursor.execute(
        """
        SELECT listing_count FROM listings 
        WHERE plz = ? AND maxprice = ? AND RootPropertyType = ? LIMIT 1
        """,
        (plz, maxprice, RootPropertyType),
    )
    result = cursor.fetchone()
    conn.close()
    return result[0] if result else None

# Save listing data to the database
def save_listing_data(plz, maxprice, RootPropertyType, listing_count):
    conn = sqlite3.connect(DATABASE_FILE)
    cursor = conn.cursor()
    cursor.execute(
        """
        INSERT OR REPLACE INTO listings (plz, maxprice, RootPropertyType, listing_count)
        VALUES (?, ?, ?, ?)
        """,
        (plz, maxprice, RootPropertyType, listing_count),
    )
    conn.commit()
    conn.close()

# Fetch a webpage with retries and log HTTP errors
def fetch_with_retries(url, session, plz, maxprice, RootPropertyType, retries=3, delay=2.5):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
    }
    for attempt in range(retries):
        sleep_time = random.uniform(delay, delay + 1)
        print(f"Attempt {attempt + 1}/{retries}: Waiting {sleep_time:.2f} seconds before fetching page...")
        time.sleep(sleep_time)
        response = session.get(url, headers=headers)
        if response.status_code == 200 and "Resultate" in response.text:
            return response.text
        print(f"HTTP Error {response.status_code}. Retrying...")
    print(f"Failed to fetch content after {retries} retries. Logging error.")
    save_error_log(plz, maxprice, RootPropertyType, response.status_code, url)
    return None

# Scrape the total count of listings from the page content
def scrape_page(soup):
    result_text = soup.find(string=re.compile(r'(\d+)\s+Resultate'))
    if result_text:
        match = re.search(r'(\d+)\s+Resultate', result_text)
        return int(match.group(1)) if match else 0
    return 0

# Process PLZ values and scrape listings for different maxprices and RootPropertyTypes
def process_plz_with_retries(maxprice_list, RootPropertyType_list, scrape_limit=20):
    try:
        # Fetch PLZs from the database
        plz_list = fetch_plz_from_db()
        if not plz_list:
            print("No PLZ values found in the database.")
            return

        scrape_count = 0
        session = requests.Session()  # Start initial session

        for maxprice in maxprice_list:
            for RootPropertyType in RootPropertyType_list:
                for index, plz in enumerate(plz_list):
                    clear_output(wait=True)  # Clear output to prevent clutter
                    print(f"Processing PLZ {index + 1}/{len(plz_list)}: {plz} | Max Price: {maxprice} | RootPropertyType: {RootPropertyType}")

                    # Recreate session every `scrape_limit` scrapes
                    if scrape_count > 0 and scrape_count % scrape_limit == 0:
                        print("Refreshing session to avoid detection...")
                        session = requests.Session()

                    # Check if the listing count exists in the database
                    existing_listing = listing_exists(plz, maxprice, RootPropertyType)
                    if existing_listing is not None:
                        print(f"Listing data for PLZ '{plz}', Max Price '{maxprice}', Root Property Type '{RootPropertyType}' already exists: {existing_listing}.")
                        continue

                    # Construct the request object and URL
                    requestobject = (
                        '{"DealType":10,"SiteId":0,"RootPropertyTypes":[' + str(RootPropertyType) + '],"PropertyTypes":[],"RoomsFrom":null,'
                        '"RoomsTo":null,"PriceFrom":null,"PriceTo":' + str(maxprice) + ',"LocationSearchString":"' + plz + '",'
                        '"Sort":3}'
                    )
                    base_url = "https://www.comparis.ch/immobilien/result/list"
                    query_string = f"requestobject={urllib.parse.quote(requestobject)}&page=0"
                    url = f"{base_url}?{query_string}"

                    # Fetch and scrape the page
                    page_content = fetch_with_retries(url, session, plz, maxprice, RootPropertyType)
                    if page_content:
                        soup = BeautifulSoup(page_content, "html.parser")
                        listing_count = scrape_page(soup)
                        print(f"Found {listing_count} listings for PLZ '{plz}', Max Price '{maxprice}', Root Property Type '{RootPropertyType}'.")
                        save_listing_data(plz, maxprice, RootPropertyType, listing_count)
                    else:
                        print(f"Failed to fetch data for PLZ '{plz}'. Debugging the issue...")

                    # Increment scrape count
                    scrape_count += 1

        print("\nScraping complete. All data saved to the database.")

    except Exception as e:
        print(f"An unexpected error occurred: {e}")


if __name__ == "__main__":
    # Initialize tables
    initialize_listings_table()
    initialize_error_logs_table()

    # Define maxprice and RootPropertyType lists
    maxprice_list = [500, 1000, 1500, 2000]
    RootPropertyType_list = [1, 3]  # Example RootPropertyTypes

    # Process PLZs and scrape listings
    process_plz_with_retries(maxprice_list, RootPropertyType_list)


## Create Master Table

In [ ]:
import sqlite3
import pandas as pd

# Database file
DATABASE_FILE = "locations.db"

def create_master_table():
    conn = sqlite3.connect(DATABASE_FILE)
    cursor = conn.cursor()
    
    # Drop existing master table
    cursor.execute("DROP TABLE IF EXISTS master_table")
    
    # Create the master table with dynamic listings columns
    cursor.execute("""
        CREATE TABLE master_table AS
        SELECT 
            cl.name_part_1 AS from_station,
            cl.coordinate_x,
            cl.coordinate_y,
            cl.plz,
            tr.to_station,
            tr.min_duration,
            tr.max_duration,
            (tr.min_duration + tr.max_duration) / 2 AS duration,  -- Calculating average duration
            tr.transfers,
            tr.connections,
            -- Example dynamic listings for specific maxprice and RootPropertyType
            MAX(CASE WHEN l.maxprice = 500 AND l.RootPropertyType = 1 THEN l.listing_count ELSE 0 END) AS listings_500_type_1,
            MAX(CASE WHEN l.maxprice = 500 AND l.RootPropertyType = 3 THEN l.listing_count ELSE 0 END) AS listings_500_type_3,                   
            MAX(CASE WHEN l.maxprice = 1000 AND l.RootPropertyType = 1 THEN l.listing_count ELSE 0 END) AS listings_1000_type_1,
            MAX(CASE WHEN l.maxprice = 1000 AND l.RootPropertyType = 3 THEN l.listing_count ELSE 0 END) AS listings_1000_type_3,
            MAX(CASE WHEN l.maxprice = 1500 AND l.RootPropertyType = 1 THEN l.listing_count ELSE 0 END) AS listings_1500_type_1,
            MAX(CASE WHEN l.maxprice = 1500 AND l.RootPropertyType = 3 THEN l.listing_count ELSE 0 END) AS listings_1500_type_3,
            MAX(CASE WHEN l.maxprice = 2000 AND l.RootPropertyType = 1 THEN l.listing_count ELSE 0 END) AS listings_2000_type_1,
            MAX(CASE WHEN l.maxprice = 2000 AND l.RootPropertyType = 3 THEN l.listing_count ELSE 0 END) AS listings_2000_type_3,
            CASE
                WHEN tr.to_station LIKE '%ETH%' THEN 'ETH'
                WHEN tr.to_station LIKE '%UZH%' THEN 'UZH'
                WHEN tr.to_station LIKE '%PHZH%' THEN 'PHZH'
                WHEN tr.to_station LIKE '%Winter%' THEN 'ZHAW'  
                ELSE 'Other'
            END AS university
        FROM combined_locations_with_plz AS cl
        LEFT JOIN transport_results_2 AS tr  -- Updated table name
        ON cl.name_part_1 = tr.from_station
        LEFT JOIN listings AS l
        ON cl.plz = l.plz
        GROUP BY cl.name_part_1, cl.coordinate_x, cl.coordinate_y, cl.plz, tr.to_station, tr.min_duration, tr.max_duration, tr.transfers, tr.connections
    """)
    
    conn.commit()
    conn.close()
    print("Master table has been created, including dynamic listings data.")

def fetch_master_table():
    conn = sqlite3.connect(DATABASE_FILE)
    query = "SELECT * FROM master_table"
    master_df = pd.read_sql_query(query, conn)
    conn.close()
    return master_df

if __name__ == "__main__":
    # Create the master table
    create_master_table()
    
    # Fetch and display the master table
    master_table = fetch_master_table()
    print(master_table.head())
